# 分词

本笔记算是分词的一个综述性质的笔记，对各个算法都是描述，需要了解更多的内容应该搭配搜索引擎使用。

分词应该是NLP的第一步，中文的NLP就不提了，句子是必然需要分词的，对于英语场景下的NLP任务，也是需要分词的， 比如"united states"它作为一个名词实体，并不是一个可分实体。

分词算法大致可以分成两种
1. 基于字典的分词
2. 基于字的分词

归根结底，这两类方法都是在概率图上寻找最短路径。

## 基于词典的分词

### 最大匹配分词算法

In [1]:
word_dict = ["研究", "研究生", "命", "生命", "的", "起源"]
max_len = 5

def forward_max(sentence: str):
    segment = list()
    while sentence:
        try_word = sentence[0: max_len] if max_len < len(sentence) else sentence
        while try_word not in word_dict:
            if len(try_word) == 1:
                break
            try_word = try_word[: len(try_word)-1]
        segment.append(try_word)
        sentence = sentence[len(try_word):]
    return segment

def backward_max(sentence: str):
    segment = list()
    while sentence:
        try_word = sentence[-max_len:] if max_len < len(sentence) else sentence
        while try_word not in word_dict:
            if len(try_word) == 1:
                break
            try_word = try_word[1-len(try_word):]
        segment.append(try_word)
        sentence = sentence[: -len(try_word)]
        #segment.reverse()
    return segment[::-1]

def max_match(sentence):
    segment1 = forward_max(sentence)
    segment2 = backward_max(sentence)
    if len(segment1) < len(segment2):
        return segment1
    else:
        return segment2

In [2]:
max_match("研究生命的起源")

['研究', '生命', '的', '起源']

### Viterbi分词

Viterbi算法实际上是一种动态规划的算法。<br>
在使用HMM模型的时候，就会用到Viterbi算法。该算法描述如下<br>
算法输入模型$\lambda = (A, B, \pi)$以及观察序列$O = (o_1, o_1, \ldots, o_n)$<br>
初始化
$$
\begin{aligned}
& \delta_1(i) = \pi_i b_i(o_1) \\
& \psi_1(i) = 0
\end{aligned}
$$
递推其中的$t = 2, 3, \ldots, N$
$$
\begin{aligned}
& \delta_t(i) = \underset{1 \le j \le N}{max}\big[\delta_{t-1}(j)a_{ji}\big]b_i(o_t) \\
& \psi_t(i) = arg\ \underset{1 \le j \le N}{max}\big[\delta_{t-1}(j)a_{ji}\big]
\end{aligned}
$$
终止
$$
\begin{aligned}
& P^* = \underset{1 \le i \le N}{max}\delta_T(i) \\
& i_T^* = arg\ \underset{1\le i\le N}{max}\delta_T(i)
\end{aligned}
$$
最优溯回路径，对$t = T-1, T-2, \ldots, 1$
$$i_t^* = \psi_{t+1}(i_{t+1}^*)$$
求得最优溯回路径为$I = (i_1^*, i_2^*, \ldots, i_T^*)$

而使用Viterbi算法实现分词的时候，需要构造一个类似下图的graph，与使用HMM进行序列标注的方式进行分词有一定区别，sentence中的每一个字并不对应一个状态而是一条边，状态的个数实际上为$\text{len}(sentence) + 1$

![viterbi segment](imgs/viterbi_segment.png)

初始化
$$
\begin{aligned}
& \delta(0) = 0\\
& \psi(0) = 0
\end{aligned}
$$
递推其中$t = 1, 2, \ldots, len(sentence)$
$$
\begin{aligned}
& \delta(t) = \underset{0\le i\le t}{min}(\delta(i)+ weight_{i\to t})\\
& \psi(t) = arg \underset{0\le i\le t}{min}(\delta(i)+ weight_{i\to t})\\
\end{aligned}
$$
然后通过$\psi$回溯其经过的最短路径，对应的边就是分词结果

In [3]:
# 这是一个概率词典，每一个词对应的值为-log(P(word))
word_dict = {
    "经常": 2.3,
    "经": 3,
    "有": 2.3,
    "有意见": 2.3,
    "意见": 1.6, 
    "分歧": 1.6,
    "见": 3,
    "意": 3,
    "见分歧": 3,
    "分": 2.3
}

max_len = 5
max_prob = 20 # 这个词不在词典中出现的时候的概率负对数值

def viterbi_segment(sentence: str):
    sigma_list = [0] * (len(sentence)+1)
    phi_list = [0] * (len(sentence)+1) # viterbi回溯矩阵，记录到达当前状态的前一个状态
    for i in range(1, len(sentence)+1):
        temp_len = max_len if i > max_len else i
        # 设置单字在词典中没有时的处理方法
        sigma_list[i] = sigma_list[i-1] + max_prob
        phi_list[i] = i-1
        for j in range(1, temp_len+1):
            try_word = sentence[i-j:i]
            if try_word in word_dict and (word_dict[try_word] + sigma_list[i-j]) < sigma_list[i]:
                sigma_list[i] = word_dict[try_word] + sigma_list[i-j]
                phi_list[i] = i-j
    segments = list()
    index = len(sentence)
    while index != 0:
        segments.append(sentence[phi_list[index]:index])
        index = phi_list[index]
    segments.reverse()
    return segments

In [4]:
viterbi_segment("经常没有意见分歧")

['经常', '没', '有意见', '分歧']

## 基于字符的分词

基于字符的分词是将分词看成序列标注问题，把一个字标记成B(Begin), I(Inside), O(Outside), E(End), S(Single)。因此也可以看成是每个字符的分类问题，输入为每个字及其前后字所构成的特征，输出为分类标记。对于分类问题，可以用统计机器学习或神经网络的方法求解。

### 生成式模型分词算法

在分词中应用比较多的是[n-gram](language_model.ipynb)模型和[HMM](../ml/hmm.ipynb)等生成模型。

HMM模型认为在解决序列标注问题时存在两种序列，一种是观测序列，即人们显性观察到的句子，而序列标签是隐状态序列，即观测序列为X，隐状态序列是Y，因果关系为Y->X。因此要得到标注结果Y，必须对X的概率、Y的概率、P(X|Y)进行计算，即建立P(X,Y)的概率分布模型。

### 判别式模型分词算法

判别式模型主要有感知机、SVM、[CRF](../ml/crf.ipynb)、最大熵模型等。在分词任务重效果比较好的是CRF。

CRF可以看作一个无向图模型，对于给定的标注序列Y和观测序列X，对条件概率P(Y|X)进行定义，而不是对联合概率建模。CRF可以说是目前最常用的分词、词性标注和实体识别算法，它对未登陆词有很好的识别能力，但开销较大。

### 神经网络分词算法

NLP中句子通常是不定长的序列，所选循环神经网络(RNN、LSTM、GRU等)或者Transformer(BERT、RoBERT之类)这样的模型。
将他们的输出经过Softmax归一化到每个标记的概率。

目前，效果最好的是LSTM + CRF， 强强联手，地表最强。